## Imports

In [1]:
%load_ext autoreload
%autoreload 2
from Datasets.Dataset_Loaders.breast_cancer_dataset_loader import prepare_wisc_breast_cancer_dataset
from Datasets.Dataset_Loaders.kbit_parity_datset_loader import generate_k_bit_parity_dataset

## Dataset Loading Examples

In [2]:
# k-bit parity dataset generation
X, y = generate_k_bit_parity_dataset(k=4)
print(f"X shape: {X.shape}")
print(f"Y shape: {y.shape}")

X shape: torch.Size([16, 4])
Y shape: torch.Size([16, 1])


In [4]:
# Breast cancer dataset loading
X, y = prepare_wisc_breast_cancer_dataset(convert_to_tensor=True)
print(f"X shape: {X.shape}")
print(f"Y shape: {y.shape}")

X shape: torch.Size([683, 9])
Y shape: torch.Size([683])
